In [ ]:
# Hackathon 2017 - [Eat Play Work] Predictive Model


In [13]:
# Import libraries
import warnings
warnings.filterwarnings("ignore")

import datetime, time, re, os

import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder, normalize
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor



In [2]:
brand_raw = pd.read_csv(r'C:\Users\User\Downloads\Hackathon\Challenge 4 - Prediction\cost_competition_combined3.csv', encoding = 'latin')
cotton_raw = pd.read_csv(r'C:\Users\User\Downloads\Hackathon\Challenge 4 - Prediction\Economic Data - Cotton Prices by Index.csv')
min_wage_raw = pd.read_csv(r'C:\Users\User\Downloads\Hackathon\Challenge 4 - Prediction\Economic Data - Minimum Wages by Country.csv')

In [3]:
# Check if need to cleanse the dataset
files = [brand_raw, cotton_raw, min_wage_raw]

for i in range(len(files)):
    print([i, files[i].isnull().values.any(), files[i].isnull().any().any(), files[i].empty, files[i].dropna().empty])

[0, True, True, False, True]
[1, False, False, False, False]
[2, False, False, False, False]


In [4]:
# Pre-processing data
brand01 = brand_raw
brand01.fillna(0, axis = 1, inplace = True)
brand01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36313 entries, 0 to 36312
Data columns (total 30 columns):
date              36313 non-null object
year              36313 non-null object
month             36313 non-null object
day               36313 non-null object
season            36313 non-null object
season2           36313 non-null object
brand             36313 non-null object
family            36313 non-null object
category          36313 non-null object
class             36313 non-null object
sub_class         36313 non-null object
fabric            36313 non-null object
fabric_desc       36313 non-null object
fabric type       36313 non-null float64
coo               36313 non-null object
gender            36313 non-null object
cost_cur          36313 non-null object
cost_art          36313 non-null float64
cost_cm           36313 non-null float64
cost_wash         36313 non-null float64
cost_trim         36313 non-null float64
cost_fab_unit     36313 non-null float64
cost_

In [5]:
brand01.head()

,date,year,month,day,season,season2,brand,family,category,class,...,cost_trim,cost_fab_unit,cost_yy,cost_fab,cost_hanger,cost_testing,cost_other,cost_fob,cost_landing,cost_style_est
0,1/13/2015 18:12,2015,1,13,Fall 2015,Fall,R,0,0,Layette,...,11.13,0.0,0.0,51.4500,2.1,0.21,0.0,94.5000,0.0,94.5000
1,1/14/2015 13:59,2015,1,14,Fall 2015,Fall,R,0,0,Layette,...,11.13,0.0,0.0,48.3525,2.1,0.21,0.0,93.0825,0.0,93.0825
2,1/14/2015 14:01,2015,1,14,Fall 2015,Fall,R,0,0,Layette,...,11.13,0.0,0.0,57.3300,2.1,0.21,0.0,100.3800,0.0,100.3800
3,1/14/2015 14:02,2015,1,14,Fall 2015,Fall,R,0,0,Layette,...,6.93,0.0,0.0,59.9760,2.1,0.21,0.0,91.8960,0.0,91.8960
4,1/14/2015 14:03,2015,1,14,Fall 2015,Fall,R,0,0,Layette,...,6.93,0.0,0.0,58.7160,2.1,0.21,0.0,88.9560,0.0,88.9560


In [6]:
# Casting
# brand01['date'] = brand01['date'].astype('float')
brand01['season'] = brand01['season'].astype('str')
brand01['season2'] = brand01['season2'].astype('str')
brand01['brand'] = brand01['brand'].astype('str')
brand01['family'] = brand01['family'].astype('str')
brand01['category'] = brand01['category'].astype('str')
brand01['season2'] = brand01['season2'].astype('str')
brand01['brand'] = brand01['brand'].astype('str')
brand01['family'] = brand01['family'].astype('str')
brand01['class'] = brand01['class'].astype('str')
brand01['sub_class'] = brand01['sub_class'].astype('str')
brand01['fabric'] = brand01['fabric'].astype('str')
brand01['fabric_desc'] = brand01['fabric_desc'].astype('str')
brand01['cost_cur'] = brand01['cost_cur'].astype('str')
brand01['coo'] = brand01['coo'].astype('str')
brand01['gender'] = brand01['gender'].astype('str')

In [7]:
# One-Hot Encoding
case01 = brand01[['cost_style_est']]
temp01_season = pd.get_dummies(brand01['season2'], drop_first=True)
temp01_brand = pd.get_dummies(brand01['brand'], drop_first=True)
temp01_class = pd.get_dummies(brand01['class'], drop_first=True)
temp01_coo = pd.get_dummies(brand01['coo'], drop_first=True)
temp01_gender = pd.get_dummies(brand01['gender'], drop_first=True)
temp01_category = pd.get_dummies(brand01['category'], drop_first=True)
temp01_family = pd.get_dummies(brand01['family'], drop_first=True)
case01a = pd.concat([case01, temp01_family, temp01_category, temp01_season, temp01_brand, temp01_class, temp01_coo, temp01_gender], axis=1)
case01a.head()

,cost_style_est,ACCESSORIES,Apparel,SLEEPWEAR,ACCESSORIES MISC,ACCESSORY,Accessories,BAGS,BELTS,Bottoms,...,Vietnam,Boy,Boys,Girl,Girls,Men,Neutral,None,Uni,Women
0,94.5000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,93.0825,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,100.3800,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,91.8960,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,88.9560,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [8]:
# Define X, y, split data

X01a = case01a.ix[:, case01a.columns != 'cost_style_est']
y01a = case01a[['cost_style_est']]
X01atrain, X01atest, y01atrain, y01atest = train_test_split(X01a, y01a, test_size = 0.8, random_state = 0)

In [9]:
X01atrain.head()

,ACCESSORIES,Apparel,SLEEPWEAR,ACCESSORIES MISC,ACCESSORY,Accessories,BAGS,BELTS,Bottoms,COLD WEATHER,...,Vietnam,Boy,Boys,Girl,Girls,Men,Neutral,None,Uni,Women
35172,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4833,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8921,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5176,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
4074,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [17]:
# Train the model
clf = RandomForestRegressor()
model_01 = clf.fit(X01atrain, y01atrain)
print(model_01.feature_importances_)
print(model_01.predict(X01atest))

[  5.15406714e-04   3.26164180e-03   4.05801205e-05   7.54596893e-04
   2.29516353e-03   1.09032414e-03   2.47348168e-03   1.04869553e-03
   6.39203751e-03   5.90873873e-05   6.00964945e-04   1.01609143e-02
   1.39916918e-04   2.53338461e-03   2.11980306e-03   3.85088893e-03
   1.41643599e-03   1.70407983e-04   2.73578803e-02   6.72128641e-04
   9.25557503e-05   6.14384593e-05   3.41728893e-02   3.63103674e-03
   1.54859617e-11   1.20472640e-04   1.39082499e-02   7.31337255e-05
   2.86298484e-04   3.84901861e-04   2.33510843e-04   2.58910131e-03
   2.81689325e-07   6.34975117e-02   3.07965025e-02   1.42476231e-02
   1.05980330e-02   1.50253769e-02   1.18574198e-02   3.51980034e-04
   3.14794849e-01   1.81307224e-04   1.02402438e-04   5.99689448e-04
   1.48979208e-02   7.11201365e-03   9.93251864e-04   1.03703080e-04
   1.27309185e-03   2.51506265e-05   1.06204783e-04   3.75870913e-03
   3.22518211e-05   6.60570424e-06   1.79388805e-04   1.87783198e-04
   1.77801901e-07   1.22063248e-06